# Solving the travelling salesman problem for Keyworth Christmas lights
* Solving the [TSP](https://en.wikipedia.org/wiki/Travelling_salesman_problem) for quickest way to walk around the village [using Google OR-Tools](https://developers.google.com/optimization/routing/tsp) and see all the lights
* Combining the [Keyworth Community Projects page](https://www.facebook.com/189497114572953/posts/1583160051873312/?extid=0&d=n) Keyworth Christmas Trail and the [Friends of Willow Brook](http://www.willowbrook-gst.org/friends-of-willow-brook/) Winter Wonderland Windows
* Nerdiness by [@Nickopotamus](https://nickopotamus.co.uk)

In [4]:
from __future__ import division
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import requests
import json
import urllib.request
import IPython

## Display all the lights
Firstly we'll see where everything is

In [6]:
iframe = '<iframe src="https://www.google.com/maps/d/u/0/embed?mid=19TiYH4ekRraLXnt2KKTe9lhoMZw-VFbh" width="840" height="480"></iframe>'
IPython.display.HTML(iframe)

## Figure out the best route
Next step is to figure out the best route. We shall use the [Google Distance Matrix API](https://developers.google.com/optimization/routing/vrp#distance_matrix_api) to build a distance matrix between clusters of displays, then solve for the fastest route between them all.

#### Create data model that will store distance matrix
* Had to combine a few nearby locations to make the API work, as the matrix can be a [maximum of 25x25](https://developers.google.com/maps/documentation/distance-matrix/usage-and-billing)
* Selected nodes highlighted in purple on the map above

In [7]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['API_key'] = 'API-KEY-GOES-HERE'
    data['addresses'] = ['6+Willowbrook+Keyworth+NG12+UK',
                         '1+Covert+Close+Keyworth+NG12+UK',
                         '1+Wolds+Rise+Keyworth+NG12+UK',
                         '11+Bunny+Lane+Keyworth+NG12+UK',
                         '13+West+Close+Keyworth+NG12+UK',
                         '16+Lilac+Close+Keyworth+NG12+UK',
                         '2+Rancliffe+Avenue+Keyworth+NG12+UK',
                         '2+Roseland+Close+Keyworth+NG12+UK',
                         '204+Mount+Pleasant+Keyworth+NG12+UK',
                         '24+Ashley+Road+Keyworth+NG12+UK',
                         '26+Rose+Grove+Keyworth+NG12+UK',
                         '27+Walton+Drive+Keyworth+NG12+UK',
                         '29+Hayes+Road+Keyworth+NG12+UK',
                         '45+Normanton+Lane+Keyworth+NG12+UK',
                         '46+Beech+Avenue+Keyworth+NG12+UK',
                         '5+Wrights+Orchard+Keyworth+NG12+UK',
                         '51+Park+Avenue+West+Keyworth+NG12+UK',
                         '52+Spinney+Road+Keyworth+NG12+UK',
                         '54+Ashley+Road+Keyworth+NG12+UK',
                         '6+Lyncombe+Gardens+Keyworth+NG12+UK',
                         '68+Selby+Lane+Keyworth+NG12+UK',
                         '7A+Dale+Road+Keyworth+NG12+UK',
                         '84A+Manor+Road+Keyworth+NG12+UK',
                         '9+Beaumont+Close+Keyworth+NG12+UK',
                         'Keyworth+Tavern+Fairway+Keyworth+NG12+UK'
                        ]
    data['num_vehicles'] = 1
    data['depot'] = 0 # Start at Willowbrook
    return data

#### Compute distance matrix

In [8]:
def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  max_elements = 100
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests
  
  num_addresses = len(addresses) 
  max_rows = max_elements // num_addresses  # Maximum number of rows that can be computed per request
  q, r = divmod(num_addresses, max_rows)    # num_addresses = q * max_rows + r
  dest_addresses = addresses
  distance_matrix = []
  
  for i in range(q):                        # Send q requests, returning max_rows rows per request.
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  if r > 0:                                  # Get the remaining remaining r rows, if necessary.
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  return distance_matrix

#### Build and send API requests

In [9]:
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):     # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.request.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

#### Build the distance matrix

In [10]:
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

#### Add distance matrix to model

In [11]:
data = create_data_model()
addresses = data['addresses']
API_key = data['API_key']
data['distance_matrix'] = create_distance_matrix(data)
print(data['distance_matrix'])

[[0, 1281, 1453, 1497, 1699, 288, 2438, 1817, 603, 1333, 1668, 1661, 2110, 1424, 504, 1809, 2028, 2491, 1235, 1426, 801, 1453, 1928, 2435, 745], [1281, 0, 415, 1488, 1690, 1398, 1379, 1808, 1257, 842, 790, 622, 1449, 547, 1143, 2007, 2019, 1432, 1069, 388, 1152, 1287, 1428, 1375, 759], [1197, 415, 0, 1155, 1357, 1065, 1046, 1475, 724, 509, 805, 289, 1116, 719, 810, 1674, 1686, 1099, 736, 159, 819, 954, 1095, 1042, 426], [1497, 1726, 1194, 0, 202, 1682, 957, 320, 1740, 660, 1323, 767, 613, 1162, 1119, 667, 531, 803, 562, 1271, 696, 562, 431, 1185, 909], [1699, 1928, 1396, 202, 0, 1884, 849, 399, 1942, 862, 1525, 969, 505, 1364, 1321, 869, 610, 694, 764, 1473, 898, 764, 322, 1128, 1111], [288, 1569, 1065, 1682, 1884, 0, 1974, 2002, 497, 1227, 1956, 1058, 2044, 1712, 398, 1995, 2214, 2027, 1129, 1143, 987, 1347, 2114, 1971, 639], [2456, 1578, 1046, 957, 849, 1920, 0, 1155, 1694, 837, 1175, 1193, 642, 1014, 1665, 1665, 950, 626, 988, 1123, 1694, 613, 622, 853, 1281], [1817, 2046, 1514, 320

### Create the routing model

In [12]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

### Create the distance callback

In [13]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

### Set the cost of travel

In [14]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
# Set to simply the distance

### Set search parameters
* Using cheapest arc - [but there are other options](https://developers.google.com/optimization/routing/routing_options#first_sol_options)

In [15]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

### Solve and print the solution

In [16]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}m'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 12276m
Route for vehicle 0:
 0 -> 8 -> 2 -> 19 -> 1 -> 11 -> 18 -> 9 -> 13 -> 10 -> 23 -> 6 -> 21 -> 3 -> 4 -> 22 -> 12 -> 17 -> 16 -> 7 -> 15 -> 20 -> 24 -> 14 -> 5 -> 0



### Save routes to an array for further processing

In [17]:
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

### Print order of route

In [18]:
routes = get_routes(solution, routing, manager)

for i in range(len(routes[0])):
    print(str(i) + ": " + data['addresses'][(routes[0][i])])

0: 6+Willowbrook+Keyworth+NG12+UK
1: 204+Mount+Pleasant+Keyworth+NG12+UK
2: 1+Wolds+Rise+Keyworth+NG12+UK
3: 6+Lyncombe+Gardens+Keyworth+NG12+UK
4: 1+Covert+Close+Keyworth+NG12+UK
5: 27+Walton+Drive+Keyworth+NG12+UK
6: 54+Ashley+Road+Keyworth+NG12+UK
7: 24+Ashley+Road+Keyworth+NG12+UK
8: 45+Normanton+Lane+Keyworth+NG12+UK
9: 26+Rose+Grove+Keyworth+NG12+UK
10: 9+Beaumont+Close+Keyworth+NG12+UK
11: 2+Rancliffe+Avenue+Keyworth+NG12+UK
12: 7A+Dale+Road+Keyworth+NG12+UK
13: 11+Bunny+Lane+Keyworth+NG12+UK
14: 13+West+Close+Keyworth+NG12+UK
15: 84A+Manor+Road+Keyworth+NG12+UK
16: 29+Hayes+Road+Keyworth+NG12+UK
17: 52+Spinney+Road+Keyworth+NG12+UK
18: 51+Park+Avenue+West+Keyworth+NG12+UK
19: 2+Roseland+Close+Keyworth+NG12+UK
20: 5+Wrights+Orchard+Keyworth+NG12+UK
21: 68+Selby+Lane+Keyworth+NG12+UK
22: Keyworth+Tavern+Fairway+Keyworth+NG12+UK
23: 46+Beech+Avenue+Keyworth+NG12+UK
24: 16+Lilac+Close+Keyworth+NG12+UK
25: 6+Willowbrook+Keyworth+NG12+UK


In [36]:
route_str = ''

for i in range(len(routes[0])-1):
    extract = data['addresses'][(routes[0][i])]
    route_str += extract + '/'

request = "https://google.com/maps/dir/" + route_str

print(request)

https://google.com/maps/dir/6+Willowbrook+Keyworth+NG12+UK/204+Mount+Pleasant+Keyworth+NG12+UK/1+Wolds+Rise+Keyworth+NG12+UK/6+Lyncombe+Gardens+Keyworth+NG12+UK/1+Covert+Close+Keyworth+NG12+UK/27+Walton+Drive+Keyworth+NG12+UK/54+Ashley+Road+Keyworth+NG12+UK/24+Ashley+Road+Keyworth+NG12+UK/45+Normanton+Lane+Keyworth+NG12+UK/26+Rose+Grove+Keyworth+NG12+UK/9+Beaumont+Close+Keyworth+NG12+UK/2+Rancliffe+Avenue+Keyworth+NG12+UK/7A+Dale+Road+Keyworth+NG12+UK/11+Bunny+Lane+Keyworth+NG12+UK/13+West+Close+Keyworth+NG12+UK/84A+Manor+Road+Keyworth+NG12+UK/29+Hayes+Road+Keyworth+NG12+UK/52+Spinney+Road+Keyworth+NG12+UK/51+Park+Avenue+West+Keyworth+NG12+UK/2+Roseland+Close+Keyworth+NG12+UK/5+Wrights+Orchard+Keyworth+NG12+UK/68+Selby+Lane+Keyworth+NG12+UK/Keyworth+Tavern+Fairway+Keyworth+NG12+UK/46+Beech+Avenue+Keyworth+NG12+UK/16+Lilac+Close+Keyworth+NG12+UK/


In [40]:
iframe = '<iframe src="https://www.google.com/maps/embed?pb=!1m166!1m12!1m3!1d19264.857024478308!2d-1.1024184141140336!3d52.87448536248946!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!4m151!3e6!4m5!1s0x4879c4b50335b869%3A0x65f36ccbedac74a!2s6%20Willow%20Brook%2C%20Keyworth%2C%20Nottingham%20NG12%205BB!3m2!1d52.8713279!2d-1.0733625!4m5!1s0x4879c4b38e73061b%3A0x21a91c7bd5c0e3fa!2s204%20Mount%20Pleasant%2C%20Keyworth%2C%20Nottingham!3m2!1d52.8736295!2d-1.077149!4m5!1s0x4879c4b1d0338939%3A0xe0877f487ee5f7e!2s1%20Wolds%20Rise%2C%20Keyworth%2C%20Nottingham%20NG12%205FU!3m2!1d52.876779!2d-1.0825854!4m5!1s0x4879c4afd24359b9%3A0x565a97e7834a7272!2s6%20Lyncombe%20Gardens%2C%20Keyworth%2C%20Nottingham%20NG12%205FZ!3m2!1d52.877460299999996!2d-1.0839896!4m5!1s0x4879c4b04c308abb%3A0x38f2e92c092917f9!2s1%20Covert%20Cl%2C%20Keyworth%2C%20Nottingham%20NG12%205GB!3m2!1d52.877555699999995!2d-1.0812089999999999!4m5!1s0x4879c4ae67bbef47%3A0x96c2334a640bff9d!2s27%20Walton%20Dr%2C%20Keyworth%2C%20Nottingham%20NG12%205FN!3m2!1d52.875797399999996!2d-1.0857297!4m5!1s0x4879c4ac3e777ea9%3A0x9117c6273a6789c1!2s54%20Ashley%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205FH!3m2!1d52.8735158!2d-1.0877425!4m5!1s0x4879c4ae94347701%3A0x7223d4da3b803ce8!2s24%20Ashley%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205FJ!3m2!1d52.875755399999996!2d-1.087647!4m5!1s0x4879c4affd970aa7%3A0xfb72ffe21fc10bce!2s45%20Normanton%20Ln%2C%20Keyworth%2C%20Nottingham!3m2!1d52.8792641!2d-1.0837525!4m5!1s0x4879c4af85851867%3A0xc39325e9fae96e6a!2s26%20Rose%20Grove%2C%20Keyworth%2C%20Nottingham%20NG12%205HE!3m2!1d52.8799467!2d-1.0866433!4m5!1s0x4879c4a6175fcdab%3A0xdf79e112165e04ac!2s9%20Beaumont%20Cl%2C%20Keyworth%2C%20Nottingham%20NG12%205JJ!3m2!1d52.880556!2d-1.0909936999999998!4m5!1s0x4879c4a825626cb9%3A0x2d50010cc0e8f330!2s2%20Rancliffe%20Ave%2C%20Keyworth%2C%20Nottingham%20NG12%205HY!3m2!1d52.878516999999995!2d-1.0950396!4m5!1s0x4879c4a9447bffff%3A0xebb8606a47f95228!2s7a%20Dale%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205HS!3m2!1d52.8753646!2d-1.0906898999999999!4m5!1s0x4879c4ab09db43f5%3A0xabf2b6282c0fef4b!2s11%20Bunny%20Ln%2C%20Keyworth%2C%20Nottingham%20NG12%205JU!3m2!1d52.8717799!2d-1.0914863!4m5!1s0x4879c4aa452f9d1b%3A0x8138c75e1e4c1fb4!2s13%20West%20Cl%2C%20Keyworth%2C%20Nottingham%20NG12%205GQ!3m2!1d52.8725082!2d-1.0935134!4m5!1s0x4879c4a9752be2b5%3A0x52bb27da6776995f!2s84A%20Manor%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205LR!3m2!1d52.8747556!2d-1.0924211!4m5!1s0x4879c4a9e82f78d1%3A0xd1eb1cdb12bb1a71!2s29%20Hayes%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205LJ!3m2!1d52.875692699999995!2d-1.0949616!4m5!1s0x4879c4a9d79cfed5%3A0x1316aa39219c259!2s52%20Spinney%20Rd%2C%20Keyworth%2C%20Nottingham%20NG12%205LN!3m2!1d52.876728799999995!2d-1.0962319999999999!4m5!1s0x4879c355ee8395bf%3A0x717e27094ea9e3ba!2s51%20Park%20Ave%20W%2C%20Keyworth%2C%20Nottingham%20NG12%205JY!3m2!1d52.8734653!2d-1.096809!4m5!1s0x4879c4aa94f3ce63%3A0x4003e594e4b3a30d!2s2%20Roseland%20Cl%2C%20Keyworth%2C%20Nottingham%20NG12%205LQ!3m2!1d52.870683!2d-1.0941912999999999!4m5!1s0x4879c4ab18bb5d1f%3A0x584728a389e85b77!2s5%20Wrights%20Orchard%2C%20Keyworth%2C%20Nottingham%20NG12%205RE!3m2!1d52.870519099999996!2d-1.0920246!4m5!1s0x4879c4b2bb6bc895%3A0xc9dc8b756ad2f5e1!2s68%20Selby%20Ln%2C%20Keyworth%2C%20Nottingham%20NG12%205AJ!3m2!1d52.8701138!2d-1.0823549!4m5!1s0x4879c4b30975059b%3A0xe310dd363b86de82!2sKeyworth%20Tavern%2C%20Fairway%2C%20Keyworth%2C%20Nottingham%20NG12%205DW!3m2!1d52.873934899999995!2d-1.0820203!4m5!1s0x4879c4b3111d252f%3A0x268ace4b3fb1978f!2s46%20Beech%20Ave%2C%20Keyworth%2C%20Nottingham%20NG12%205DJ!3m2!1d52.8709752!2d-1.0790218!4m5!1s0x4879c4b4bbfe7f45%3A0xece02e5c2e970aae!2s16%20Lilac%20Cl%2C%20Keyworth%2C%20Nottingham%20NG12%205DN!3m2!1d52.8702664!2d-1.0755838!5e0!3m2!1sen!2suk!4v1607345586851!5m2!1sen!2suk" width="840" height="450" frameborder="0" style="border:0;" allowfullscreen="" aria-hidden="false" tabindex="0"></iframe>'
IPython.display.HTML(iframe)